In [1]:
# No leak data
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


In [2]:
# Bayesian Optimization Functions

# https://www.kaggle.com/aitude/ashrae-hyperparameter-tuning

In [3]:
train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')

In [4]:
# some feature enginnering

train_df['age'] = train_df['year_built'].max()-train_df['year_built']+1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

#Handling missing values
# To streamline this though process it is useful to know the 3 categories in which missing data can be classified into:

# Missing Completely at Random (MCAR)
# Missing at Random (MAR)
# Missing Not at Random (MNAR)

train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 

train_df['square_feet'] = np.log1p(train_df['square_feet'])
test_df['square_feet'] = np.log1p(test_df['square_feet'])

In [5]:
def make_is_bad_zero(Xy_subset, min_interval=48, summer_start=3000, summer_end=7500):
    """Helper routine for 'find_bad_zeros'.
    
    This operates upon a single dataframe produced by 'groupby'. We expect an 
    additional column 'meter_id' which is a duplicate of 'meter' because groupby 
    eliminates the original one."""
    meter = Xy_subset.meter_id.iloc[0]
    is_zero = Xy_subset.meter_reading == 0
    if meter == 0:
        # Electrical meters should never be zero. Keep all zero-readings in this table so that
        # they will all be dropped in the train set.
        return is_zero

    transitions = (is_zero != is_zero.shift(1))
    all_sequence_ids = transitions.cumsum()
    ids = all_sequence_ids[is_zero].rename("ids")
    if meter in [2, 3]:
        # It's normal for steam and hotwater to be turned off during the summer
        keep = set(ids[(Xy_subset.timestamp < summer_start) |
                       (Xy_subset.timestamp > summer_end)].unique())
        is_bad = ids.isin(keep) & (ids.map(ids.value_counts()) >= min_interval)
    elif meter == 1:
        time_ids = ids.to_frame().join(Xy_subset.timestamp).set_index("timestamp").ids
        is_bad = ids.map(ids.value_counts()) >= min_interval

        # Cold water may be turned off during the winter
        jan_id = time_ids.get(0, False)
        dec_id = time_ids.get(8283, False)
        if (jan_id and dec_id and jan_id == time_ids.get(500, False) and
                dec_id == time_ids.get(8783, False)):
            is_bad = is_bad & (~(ids.isin(set([jan_id, dec_id]))))
    else:
        raise Exception(f"Unexpected meter type: {meter}")

    result = is_zero.copy()
    result.update(is_bad)
    return result

def find_bad_zeros(X, y):
    """Returns an Index object containing only the rows which should be deleted."""
    Xy = X.assign(meter_reading=y, meter_id=X.meter)
    is_bad_zero = Xy.groupby(["building_id", "meter"]).apply(make_is_bad_zero)
    return is_bad_zero[is_bad_zero].index.droplevel([0, 1])

def find_bad_sitezero(X):
    """Returns indices of bad rows from the early days of Site 0 (UCF)."""
    return X[(X.timestamp < 3378) & (X.site_id == 0) & (X.meter == 0)].index

def find_bad_building1099(X, y):
    """Returns indices of bad rows (with absurdly high readings) from building 1099."""
    return X[(X.building_id == 1099) & (X.meter == 2) & (y > 3e4)].index

def find_bad_rows(X, y):
    return find_bad_zeros(X, y).union(find_bad_sitezero(X)).union(find_bad_building1099(X, y))


In [6]:
# bad rows 삭제 3% 
train_df['timestamp'] = (train_df.timestamp - pd.to_datetime("2016-01-01")).dt.total_seconds() // 3600
rows = find_bad_rows(train_df.drop(['meter_reading'],axis =1 ),train_df['meter_reading'])
rows = list(rows)
train_df= train_df.drop(rows)

In [7]:
# drop_cols = ['date',"precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed","timestamp"]
drop_cols = ['date',"timestamp"]
target = train_df["meter_reading_log1p"]
del train_df["meter_reading"], train_df['meter_reading_log1p']
train_df = train_df.drop(drop_cols, axis=1)
drop_cols += ["row_id"]
# drop_cols.remove('date')
test_df = test_df.drop(drop_cols, axis=1)

In [8]:
train_df.head()
categorical_features = ["building_id", "site_id", "meter", "hour", "weekend",'month','is_holiday','primary_use']

In [9]:
leak_df = pd.read_pickle('leak_df.pkl')
leak_df = leak_df.dropna()


In [10]:
%%time
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from bayes_opt import BayesianOptimization
folds = 3
seed = 99 #666

# Hyperparameter range
params_range = {
                'num_leaves': (1000, 1500),
                'feature_fraction': (0.7, 0.9),
                'bagging_fraction': (0.8, 1),
                'max_depth': (10, 11),
                'lambda_l1': (0, 5),
                'lambda_l2': (0, 5),
                'min_split_gain': (0.001, 0.1),
                'min_child_weight': (5, 50),
                'learning_rate' : (.03,.07)
               }



Wall time: 1.28 s


In [ ]:
%%time
paramlst = []
scorelst =[]
def running(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight, learning_rate):
    params = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": int(round(num_leaves)),
    "learning_rate": learning_rate,
    "feature_fraction": max(min(feature_fraction,1),0),
    "bagging_fraction": max(min(bagging_fraction,1),0),
    "max_depth" : int(round(max_depth)),
    "lambda_l1" : max(lambda_l1,0),
    "lambda_l2" : max(lambda_l2,0),
    "min_split_gain": min_split_gain,
    "min_child_weight": min_child_weight,
    "metric": "rmse",
    "n_jobs" : 10
    }
    
    print(params)
    kf = KFold(n_splits=folds, shuffle=False, random_state=seed)
    scores = [] 
    models = []
    for train_index, val_index in kf.split(train_df):
        train_X = train_df.iloc[train_index]
    
        val_X = train_df.iloc[val_index]
        train_y = target.iloc[train_index]
        val_y = target.iloc[val_index]
        lgb_train = lgb.Dataset(train_X, train_y,categorical_feature=categorical_features)
        lgb_eval = lgb.Dataset(val_X, val_y,categorical_feature=categorical_features)
        gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=2000, #300,
                    valid_sets=(lgb_train, lgb_eval),
#                     feval=rmsle,
                    early_stopping_rounds= 50,#100,
                    verbose_eval=30) #100)
        models.append(gbm)
        del gbm
    
    i = 0
    res3 = []
    step_size = 50000
        
    for j in tqdm(range(int(np.ceil(test_df.shape[0] / 50000)))):
        res3.append([np.expm1(model.predict(test_df.iloc[i:i + step_size])) for model in models])
        i += step_size
        
    res4 = []
    print(len(res3))
    for k in range(len(res3)):
        res4.append((0.6*res3[k][0]+0.3*res3[k][1]+0.1*res3[k][2]))
#         if i==100 : break
    
    
    res4 = np.concatenate(res4)
    sample_submission = pd.read_csv('sample_submission.csv')
    sample_submission["meter_reading"] = res4
    sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0

    sample_submission = pd.merge(sample_submission,leak_df.loc[:,['row_id','leak_meter_reading']],how ='inner')    
    score = np.sqrt(mean_squared_log_error( sample_submission['leak_meter_reading'], sample_submission['meter_reading'] ))
    print(score)
    scorelst.append(score)
    paramlst.append(params)
    
    df_para = pd.DataFrame({"parameter" : paramlst,"score": scorelst})
    df_para.to_csv('para.csv',index=False)
    
    return -np.sqrt(mean_squared_log_error( sample_submission['leak_meter_reading'], sample_submission['meter_reading'] ))        

    
Baysian = BayesianOptimization(running, params_range,random_state = seed)
Baysian.maximize(init_points = 5, n_iter = 20)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------
{'objective': 'regression', 'boosting': 'gbdt', 'num_leaves': 1495, 'learning_rate': 0.062321998534593914, 'feature_fraction': 0.7976156798481168, 'bagging_fraction': 0.9344557117261584, 'max_depth': 11, 'lambda_l1': 4.127475870179482, 'lambda_l2': 0.15723193813149072, 'min_split_gain': 0.005622876330188372, 'min_child_weight': 18.393012441677477, 'metric': 'rmse', 'n_jobs': 10}


C:\Users\yseon\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[30]	training's rmse: 1.02813	valid_1's rmse: 1.17218
[60]	training's rmse: 0.738094	valid_1's rmse: 0.97515
[90]	training's rmse: 0.63781	valid_1's rmse: 0.917823
[120]	training's rmse: 0.610505	valid_1's rmse: 0.904778
[150]	training's rmse: 0.591082	valid_1's rmse: 0.897663
[180]	training's rmse: 0.575879	valid_1's rmse: 0.895148
[210]	training's rmse: 0.562561	valid_1's rmse: 0.892409
[240]	training's rmse: 0.550937	valid_1's rmse: 0.890725
[270]	training's rmse: 0.538694	valid_1's rmse: 0.889334
[300]	training's rmse: 0.528497	valid_1's rmse: 0.889293
[330]	training's rmse: 0.520405	valid_1's rmse: 0.888592
[360]	training's rmse: 0.513929	valid_1's rmse: 0.888774
[390]	training's rmse: 0.507213	valid_1's rmse: 0.889234
Early stopping, best iteration is:
[343]	training's rmse: 0.517548	valid_1's rmse: 0.888169
Training until validation scores don't improve for 50 rounds
[30]	training's rmse: 0.918191	valid_1's rmse: 1.137

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [38:30<00:00,  2.90s/it]


834
0.940564196078774
|  1        | -0.9406   |  0.9345   |  0.7976   |  4.127    |  0.1572   |  0.06232  |  10.57    |  18.39    |  0.005623 |  1.495e+0 |
{'objective': 'regression', 'boosting': 'gbdt', 'num_leaves': 1262, 'learning_rate': 0.049765898089639035, 'feature_fraction': 0.853958605637988, 'bagging_fraction': 0.801365146608731, 'max_depth': 11, 'lambda_l1': 3.7338355049714984, 'lambda_l2': 1.887194681674826, 'min_split_gain': 0.09742167338606704, 'min_child_weight': 22.795431995561913, 'metric': 'rmse', 'n_jobs': 10}


...

Training until validation scores don't improve for 50 rounds
[30]	training's rmse: 1.32371	valid_1's rmse: 1.40448
[60]	training's rmse: 1.05117	valid_1's rmse: 1.19247
[90]	training's rmse: 0.879284	valid_1's rmse: 1.06768
[120]	training's rmse: 0.79897	valid_1's rmse: 1.01198
[150]	training's rmse: 0.726561	valid_1's rmse: 0.964357
[180]	training's rmse: 0.68679	valid_1's rmse: 0.938854
[210]	training's rmse: 0.656732	valid_1's rmse: 0.92142
[240]	training's rmse: 0.640978	valid_1's rmse: 0.911715
[270]	training's rmse: 0.629627	valid_1's rmse: 0.906744
[300]	training's rmse: 0.619124	valid_1's rmse: 0.901989
[330]	training's rmse: 0.607994	valid_1's rmse: 0.897115
[360]	training's rmse: 0.599543	valid_1's rmse: 0.8938
[390]	training's rmse: 0.592003	valid_1's rmse: 0.891411
[420]	training's rmse: 0.586443	valid_1's rmse: 0.889763
[450]	training's rmse: 0.581892	valid_1's rmse: 0.888058
[480]	training's rmse: 0.576726	valid_1's rmse: 0.886504
[510]	training's rmse: 0.571719	valid_1's

100%|██████████████████████████████████████████████████████████████████████████████| 834/834 [2:53:21<00:00, 12.28s/it]


834
0.9398112731176935
|  13       | -0.9398   |  0.9796   |  0.7061   |  3.101    |  0.807    |  0.03043  |  10.64    |  49.66    |  0.05456  |  1.001e+0 |
{'objective': 'regression', 'boosting': 'gbdt', 'num_leaves': 1000, 'learning_rate': 0.055861921192162954, 'feature_fraction': 0.7289323188494812, 'bagging_fraction': 0.8332483904053708, 'max_depth': 11, 'lambda_l1': 4.803761551265161, 'lambda_l2': 1.1019391559327336, 'min_split_gain': 0.0040384139597368085, 'min_child_weight': 49.624027452225434, 'metric': 'rmse', 'n_jobs': 10}


C:\Users\yseon\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds
[30]	training's rmse: 1.08466	valid_1's rmse: 1.2262
[60]	training's rmse: 0.809111	valid_1's rmse: 1.02025
[90]	training's rmse: 0.683283	valid_1's rmse: 0.938566
[120]	training's rmse: 0.645469	valid_1's rmse: 0.919553
[150]	training's rmse: 0.620703	valid_1's rmse: 0.908019
[180]	training's rmse: 0.606488	valid_1's rmse: 0.901433
[210]	training's rmse: 0.59244	valid_1's rmse: 0.897098
[240]	training's rmse: 0.580976	valid_1's rmse: 0.894933
[270]	training's rmse: 0.571205	valid_1's rmse: 0.892773
[300]	training's rmse: 0.561024	valid_1's rmse: 0.89143
[330]	training's rmse: 0.551261	valid_1's rmse: 0.890577
[360]	training's rmse: 0.542877	valid_1's rmse: 0.890116
[390]	training's rmse: 0.535084	valid_1's rmse: 0.88984
[420]	training's rmse: 0.530216	valid_1's rmse: 0.889657
[450]	training's rmse: 0.525496	valid_1's rmse: 0.889884
Early stopping, best iteration is:
[423]	training's rmse: 0.529695	valid_1's rmse: 0.889527
Tr

In [12]:
# def lgb_best_params(X, y,opt_params,init_points=2, optimization_round=20, n_folds=3, random_seed=0, cv_estimators=1000):
    
#     # prepare dataset
#     categorical_features = ["building_id", "site_id", "meter", "primary_use", "is_holiday", "weekend"]
#     train_data = lgb.Dataset(data=X, label=y, categorical_feature = categorical_features, free_raw_data=False)
    
#     def lgb_run(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight,learning_rate):
#         params = {"boosting": "gbdt",'application':'regression','num_iterations':cv_estimators, 'early_stopping_round':int(cv_estimators/5), 'metric':'rmse'}
#         params["num_leaves"] = int(round(num_leaves))
#         params['feature_fraction'] = max(min(feature_fraction, 1), 0)
#         params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
#         params['max_depth'] = int(round(max_depth))
#         params['lambda_l1'] = max(lambda_l1, 0)
#         params['lambda_l2'] = max(lambda_l2, 0)
#         params['min_split_gain'] = min_split_gain
#         params['min_child_weight'] = min_child_weight
#         params['learning_rate'] = learning_rate
#         cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=False, verbose_eval =cv_estimators, metrics=['rmse'])
#         return -min(cv_result['rmse-mean'])
    
#     params_finder = BayesianOptimization(lgb_run, opt_params, random_state=2021)
#     # optimize
#     params_finder.maximize(init_points=init_points, n_iter=optimization_round)

#     # return best parameters
#     return params_finder.max

In [13]:
# #  number of fold
# fold = 3

# # Hyperparameter range
# params_range = {
#                 'num_leaves': (1000, 1280),
#                 'feature_fraction': (0.7, 0.9),
#                 'bagging_fraction': (0.8, 1),
#                 'max_depth': (10, 11),
#                 'lambda_l1': (2, 5),
#                 'lambda_l2': (2, 5),
#                 'min_split_gain': (0.001, 0.1),
#                 'min_child_weight': (5, 50),
#                 'learning_rate' : (.05,.07)
#                }

# # You can experiments with different estimators in a single execution. I'm using small numbers for demonstration purpose so you must change to high numbers.
# cv_estimators = [50,100,200]

# #n_iter: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
# optimization_round = 10 # Simply, 10 models with different parameter will be tested. And the best one will be returned.

# #init_points: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.
# init_points = 2

# random_seed = 2010

In [14]:
# best_params= []
# for cv_estimator in cv_estimators:
#     opt_params = lgb_best_params(features, target,params_range, init_points=init_points, optimization_round=optimization_round, n_folds=fold, random_seed=random_seed, cv_estimators=cv_estimator)
#     opt_params['params']['iteration'] = cv_estimator
#     opt_params['params']['fold'] = fold
#     opt_params['params']['rmse'] = opt_params['target']
#     best_params.append(opt_params['params'])

In [15]:
# df_params = pd.DataFrame(best_params).reset_index()
# df_params = df_params[['iteration','fold','num_leaves','learning_rate','bagging_fraction',
#  'feature_fraction',
#  'lambda_l1',
#  'lambda_l2',
#  'max_depth',
#  'min_child_weight',
#  'min_split_gain',
#  'rmse']]
# df_params.to_csv('best_params.csv')